## Heart Disease Predictor

In [1]:
# Import the libraries
import numpy as np # To work with arrays
import pandas as pd # To work with the data
# Importing Sk learn libraries/ Modules
from sklearn.model_selection import train_test_split # Module to split the data into training and testing
from sklearn.linear_model import LogisticRegression # To build the model
from sklearn.metrics import accuracy_score # Evaluation

In [2]:
# Improting the data

df = pd.read_csv("heart_disease_data.csv")

In [3]:
# First 5 Records
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [4]:
df.tail()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0
302,57,0,1,130,236,0,0,174,0,0.0,1,1,2,0


In [5]:
# Getting the dimension of our data
df.shape

(303, 14)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


In [7]:
# missing values
df.isnull().sum()

,0
age,0
sex,0
cp,0
trestbps,0
chol,0
fbs,0
restecg,0
thalach,0
exang,0
oldpeak,0


In [8]:
# Descriptive statistics
df.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531,0.544554
std,9.082101,0.466011,1.032052,17.538143,51.830751,0.356198,0.525860,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277,0.498835
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,55.000000,1.000000,1.000000,130.000000,240.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,274.500000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In [9]:
# Count the number of occurrences of each unique value in the 'target' column
# ----------------------------------------------------------------------------
# df['target'] → Selects the 'target' column from the DataFrame.
# .value_counts() → Counts how many times each unique value appears.
# This is useful to:
#   - Check the distribution of classes (e.g., balanced vs. imbalanced dataset)
#   - Understand if one class has much more data than the other (which can bias the model)
df['target'].value_counts()


,count
target,
1,165
0,138


- 1 $\rightarrow$ Heart Disease
- 0 $\rightarrow$ Healthy patient.

#### Splitting Features and Target

In [10]:
# Create the feature set (X) by removing the target column
# ---------------------------------------------------------
# df.drop(columns='target', axis=1):
#   - columns='target' → specifies which column(s) to drop; here, we drop the 'target' column
#   - axis=1 → tells Pandas we are dropping a column (axis=0 would drop rows)
# Why?
#   - We only want the input features in X (independent variables)
#   - The 'target' column is our label (Y), so it should not be part of the input features
X = df.drop(columns='target', axis=1)

# Display the feature data
# -------------------------
# This will print all columns except 'target'
# Example (for heart disease dataset):
#   age, sex, chest_pain_type, resting_bp, cholesterol, ...
X


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3


In [11]:
# Extract the target column (labels) from the DataFrame
# ------------------------------------------------------
# df['target'] → Selects the 'target' column from the DataFrame df.
# This column contains the output variable (labels) we want to predict.
# Example: In a heart disease dataset, 'target' might be:
#   0 → No heart disease
#   1 → Heart disease present
Y = df['target']

# Display the target values
# --------------------------
# Printing Y will show all the values from the 'target' column,
# usually as a Pandas Series.
Y


,target
0,1
1,1
2,1
3,1
4,1
...,...
298,0
299,0
300,0
301,0


#### Training and Testing

In [12]:
X_train , X_test, Y_train, Y_test = train_test_split(X, # Features (input variables) — e.g., size, color, weight
                                                     Y,  # Target labels (output variable) — e.g., 0 = Not Spam, 1 = Spam
                                                     test_size=0.2,  # 20% of the data will be used for testing, 80% for training
                                                     stratify=Y, # Ensures train and test sets have the same class distribution as the original dataset
                                                     random_state=42)  # Random seed to make the split reproducible (same split every run)

In [13]:
X.shape

(303, 13)

In [14]:
X_train.shape

(242, 13)

In [15]:
X_test.shape

(61, 13)

In [16]:
Y_train.shape

(242,)

In [17]:
Y_test.shape

(61,)

### Model Building

LogisticRegression is used to create the model where end of the it is a binary classification.

In [18]:
model = LogisticRegression()

In [19]:
# Train the machine learning model using the training data
# ---------------------------------------------------------
# model.fit() is the method that tells the model to "learn" from the data.
# X_train → The training input features (independent variables) that the model uses to find patterns.
# Y_train → The corresponding correct labels/outputs (dependent variable) for each row in X_train.
# The model adjusts its internal parameters (weights, splits, rules, etc.)
# so that it can correctly predict Y values from X values.
# After this step, the model has "learned" the relationship between inputs and outputs
# and can be used to make predictions on new, unseen data.
model.fit(X_train, Y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

### Evaluation


In [20]:
# Use the trained model to make predictions on the training set
# -------------------------------------------------------------
# model.predict(X_train) → Passes the training feature data (X_train)
# through the trained model to get predicted class labels for each row.
X_train_prediction = model.predict(X_train)

# Calculate how accurate the model's predictions are on the training data
# ------------------------------------------------------------------------
# accuracy_score() compares the predicted labels (X_train_prediction)
# with the actual correct labels (Y_train).
# It returns the proportion of correct predictions as a decimal value (0 to 1).
training_accuracy = accuracy_score(X_train_prediction, Y_train)

# Display the training accuracy in a readable format
# ---------------------------------------------------
# f-string formatting is used to insert the accuracy value into the output message.
# The result shows how well the model learned from the training data.
print(f" The training accuracy is {training_accuracy}")


 The training accuracy is 0.8429752066115702


In [21]:
# Evaluate model performance on the testing data
# ===============================================

# 1. Make predictions on the testing set
# ---------------------------------------
# model.predict(X_test) → Uses the trained model to predict labels for X_test
# (data the model has never seen before).
# This step checks how well the model generalizes to new, unseen data.
X_test_prediction = model.predict(X_test)

# 2. Calculate accuracy on the test set
# --------------------------------------
# accuracy_score() compares the predicted labels (X_test_prediction)
# with the actual correct labels (Y_test).
# The result is the percentage of correct predictions (value between 0 and 1).
testing_accuracy = accuracy_score(X_test_prediction, Y_test)

# 3. Print the testing accuracy
# -----------------------------
# f-string formatting is used to embed the accuracy value into the output message.
# This tells us how well the model performs on completely new data.
print(f" The testing accuracy is {testing_accuracy}")


 The testing accuracy is 0.8032786885245902


### Building the Predicting system :

In [22]:
# Tuple of the feature data --> This is our input for prediction
# ----------------------------------------------------------------
# Each value in the tuple represents one feature for the patient
# Example: (age, gender, chest pain type, resting BP, cholesterol, etc.)
# This data must match the same order and format as the training features.
input_data = (51, 1, 3, 125, 213, 0, 0, 125, 1, 1.4, 2, 1, 2)

# Convert the tuple into a NumPy array
# -------------------------------------
# model.predict() works with array-like structures (NumPy arrays, DataFrames).
# Converting to NumPy array ensures compatibility with scikit-learn.
input_data = np.asarray(input_data)

# Reshape the array to match the model's expected input shape
# ------------------------------------------------------------
# The model expects a 2D array: (number_of_samples, number_of_features)
# Currently, input_data is 1D (just one row of data).
# reshape(1, -1) → "1" means 1 sample, "-1" means infer the number of features automatically.
input_data = input_data.reshape(1, -1)

# Use the trained model to make a prediction for this input
# ----------------------------------------------------------
# model.predict() returns the predicted class (0 or 1 in this case)
# 0 → No heart disease, 1 → Heart disease present
prediction = model.predict(input_data)

# Display prediction result in human-readable form
# -------------------------------------------------
# prediction[0] extracts the predicted class from the array.
# Based on the class, display an appropriate message.
if prediction[0] == 0:
    print("Good News: The patient doesn't have any heart disease")
else:
    print("The Patient should visit the doctor")


The Patient should visit the doctor


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
